In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,lit,when,format_number,row_number,col
from pyspark.sql.types import StringType
from pyspark.sql.window import Window

from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.worksheet.table import Table, TableStyleInfo

spark = SparkSession.builder \
    .appName("CSV Loader") \
    .config("spark.executor.memory", "32g") \
    .config("spark.driver.memory", "32g") \
    .config("spark.executor.memoryOverhead", "8192") \
    .config("spark.driver.memoryOverhead", "8192") \
    .config("spark.executor.cores", "7") \
    .config("spark.driver.cores", "7") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/06 15:08:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
morefire_file ='/Users/maralsheikhzadeh/Documents/Codes/Analytics/morefire/hg roxxad okt24.xlsx'
df_first = pd.read_excel(morefire_file,header=7)
# df_first['ID'] = df_first['ID'].fillna(0).astype(int).astype(str)
df_first['Transaktions-ID'] = df_first['Transaktions-ID'].astype(str)
df_first['Transaktions-ID'] = df_first['Transaktions-ID'].str.replace(".0","")

## Update Originals first
With Server-connect folder **dbf_to_spark.ipynb**, first update all the csv files below so that we can find the EXT_Verweis in them. The following fields connect the data from all 4 countries into 3 unified tables to further connect them with the file from Morefire.

In [5]:
V2AD1096 = spark.read.csv('/Volumes/MARAL/CSV/F01/V2AD1096.csv',sep=';',header=True)
V2AD1056 = spark.read.csv('/Volumes/MARAL/CSV/F01/V2AD1056.csv',sep=';',header=True)
V2AD1156 = spark.read.csv('/Volumes/MARAL/CSV/F01/V2AD1156.csv',sep=';',header=True)

In [6]:
V2AD1096_FR = spark.read.csv('/Volumes/MARAL/CSV/f02/V2AD1096.csv',sep=';',header=True)
V2AD1056_FR = spark.read.csv('/Volumes/MARAL/CSV/f02/V2AD1056.csv',sep=';',header=True)
V2AD1156_FR = spark.read.csv('/Volumes/MARAL/CSV/f02/V2AD1156.csv',sep=';',header=True)

In [7]:
V2AD1096_AU = spark.read.csv('/Volumes/MARAL/CSV/F03/V2AD1096.csv',sep=';',header=True)
V2AD1056_AU = spark.read.csv('/Volumes/MARAL/CSV/F03/V2AD1056.csv',sep=';',header=True)
V2AD1156_AU = spark.read.csv('/Volumes/MARAL/CSV/F03/V2AD1156.csv',sep=';',header=True)

In [8]:
V2AD1096_CH = spark.read.csv('/Volumes/MARAL/CSV/F04/V2AD1096.csv',sep=';',header=True)
V2AD1056_CH = spark.read.csv('/Volumes/MARAL/CSV/F04/V2AD1056.csv',sep=';',header=True)
V2AD1156_CH = spark.read.csv('/Volumes/MARAL/CSV/F04/V2AD1156.csv',sep=';',header=True)

In [9]:
V2AD1096 = V2AD1096_FR.union(V2AD1096_AU).union(V2AD1096_CH).union(V2AD1096)
V2AD1056 = V2AD1056_FR.union(V2AD1056_AU).union(V2AD1056_CH).union(V2AD1056)
V2AD1156 = V2AD1156_FR.union(V2AD1156_AU).union(V2AD1156_CH).union(V2AD1156)


In [10]:
V2AD1096 = V2AD1096.withColumn('SHOPNUMMER', col('SHOPNUMMER').cast(StringType()))

In [ ]:
df = spark.createDataFrame(df_first)


df = df.join(V2AD1096[['SHOPNUMMER','RECH_NR']],df['Transaktions-ID'] == V2AD1096['SHOPNUMMER'],how='left')


In [13]:
# merged_df['RECH_NR'] = merged_df['RECH_NR'].astype(str)
df = df.withColumn('RECH_NR', col('RECH_NR').cast(StringType()))
V2AD1056 = V2AD1056.withColumn('RECH_NR', col('RECH_NR').cast(StringType()))

In [14]:
df = df.join(V2AD1056[['VERWEIS','RECH_NR','BEST_WERT','MWST1','AUF_ANLAGE']],on='RECH_NR',how='left')
df = df.join(V2AD1156[['ART_NR','PREIS','MWST','RECHNUNG']],df['RECH_NR']==V2AD1156['RECHNUNG'],how='left')

## Changing from Pyspark to Pandas
I used Pyspark to load and connect the data from our databases because the data is so large and Pandas is so slow. Now that everything is joined and the data is filtered to the original lines we had, it's easier to transform it back to Pandas to do the final analysis on it.

In [15]:
df = df.toPandas()
df_new = df.copy()


Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=38255Kb max_used=38255Kb free=92816Kb
 bounds [0x000000010a1e8000, 0x000000010c778000, 0x00000001121e8000]
 total_blobs=12857 nmethods=11926 adapters=839
 compilation: disabled (not enough contiguous free space left)


### Transforming data

- We create a columnn for items that were **Not Found** in the VS4 Data. We make a * sign for them to know what is missing from data.
- The third line removes items that were not computed in Morefire's Sales_Betrag, namely: Versandkosten (VK) and Geschenkverpackung (011P00). Items may need to be added to this list in the future depending on the items that we see further that are not computed here.
- Then we filter only the columns we need. and change data types for our requirements.
- Finally, we aggregate all the lines from the same Rechnung and Extra so that we compute the Fakturierte Nettoumsatz based on the Nettoumsatz value for each product in the list.

In [ ]:
# df_new['Publisher_ID'] = df_new['Publisher_ID'].astype(str)
df_new['Not Found'] = ""
df_new.loc[df_new['SHOPNUMMER'].isna(),'Not Found'] = '*'

df_new = df_new[df_new['ART_NR'].isin(['VK','011P00'])==False]

df_new['PREIS'] = df_new['PREIS'].astype(float).fillna(0)
df_new = df_new.drop_duplicates(subset=['Transaktions-ID','PREIS','ART_NR'])
df_grouped = df_new.groupby(
    ['Transaktions-ID']
).agg({'PREIS': 'sum','Not Found':'first'}).reset_index().rename(columns={'PREIS': 'Fakturierte Nettoumsatz'})
df_grouped = df_grouped.merge(df_first,on='Transaktions-ID',how='left')
# df_grouped = df_grouped[['Quelle/Medium','Transaktions-ID', 'Gesamtumsatz', 'Fakturierte Nettoumsatz','Not Found']]
# df_grouped['Gesamtumsatz'] = df_grouped['Gesamtumsatz'].str.replace(',','.')
df_grouped['Gesamtumsatz'] = pd.to_numeric(df_grouped['Gesamtumsatz'],errors='coerce')

Finally, we compute the difference between our nettoumsatz and what is given by Morefire as Sales_Betrag and leave it in the Difference column.
When everything is computed. we save the data in an excel sheet and there do some minor formatting like formatting the data as a table and then using formatting cell to add Euro sign to columns with Euro values and so on.

In [18]:
# df_grouped['Unterschied'] = ""
# df_grouped.loc[(df_grouped['Gesamtumsatz'] - df_grouped['Fakturierte Nettoumsatz'])>1, 'Unterschied'] = df_grouped['Gesamtumsatz'] - df_grouped['Fakturierte Nettoumsatz']

In [19]:
df_grouped

,ID,Fakturierte Nettoumsatz,Not Found,AWIN ID,Publisher_ID,Sale_Betrag,Provision,Datum,Site_Name,URL,Ablehnungs_Grund,Referrer,Publisher_URL,Transaktions_Teile,Land_nach_IP,Benutzerdef_Parameter,Produkte,getrackte_Teile,Netzwerkgebühr
0,173174065723239,99.82,,1719046774,626889,123.34,14.80,2024-11-16 08:05:00,adstrong CSS,https://preisvergleich.adstrong.de/,NaN,https://www.hagengrote.de/,https://www.google.de/,"DEFAULT:123,34",Germany,1: Julia_Grote,"[{""product_id"":""018HC46L"",""product_name"":""LED-...","DEFAULT:123,34",0.25
1,173174258928539,12.10,,1719056209,176013,12.10,0.97,2024-11-16 08:36:00,Redbrain Ltd,https://www.redbrain.shop/,NaN,https://www.hagengrote.de/,https://de.redbrain.shop/,"DEFAULT:12,10",Germany,1: Hagen_Grote,"[{""product_id"":""541T07"",""product_name"":""Super ...","DEFAULT:12,10",0.02
2,173174592276139,116.51,,1719082958,626889,116.51,13.98,2024-11-16 09:32:00,adstrong CSS,https://preisvergleich.adstrong.de/,NaN,https://www.hagengrote.de/,https://www.google.com/,"DEFAULT:116,51",Germany,1: Hagen_Grote,"[{""product_id"":""243HZ24L"",""product_name"":""Deko...","DEFAULT:116,51",0.23
3,173175016202439,63.42,,1719117931,333891,63.42,3.81,2024-11-16 10:42:00,AVS Abrechnungs- und Verwaltungs- Systeme GmbH,https://www.avs.de,NaN,https://www.hagengrote.de/,https://www.bsw.de/partner/hagen-grote/7400194...,"DEFAULT:63,42",Germany,1: Hagen_Grote,"[{""product_id"":""041U02"",""product_name"":""Feines...","DEFAULT:63,42",0.13
4,173175069515639,62.98,,1719122971,300443,62.99,3.78,2024-11-16 10:52:00,Savoo DE,http://www.savoo.de,NaN,https://www.hagengrote.de/,https://www.savoo.de/,"DEFAULT:62,99",Germany,1: Julia_Grote,"[{""product_id"":""242HU2525CR"",""product_name"":""W...","DEFAULT:62,99",0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,173429482297639,142.82,,1752233107,328571,142.82,17.14,2024-12-15 21:34:00,DeutschlandCard GmbH,https://www.deutschlandcard.de/,NaN,https://www.hagengrote.de/,NaN,"DEFAULT:142,82",Germany,1: Hagen_Grote,"[{""product_id"":""166D02"",""product_name"":""Hochbe...","DEFAULT:142,82",0.29
1311,173429535676139,46.68,,1752240322,626889,46.68,5.60,2024-12-15 21:42:00,adstrong CSS,https://preisvergleich.adstrong.de/,NaN,https://www.hagengrote.de/,https://www.google.com/,"DEFAULT:46,68",Germany,1: Hagen_Grote,"[{""product_id"":""023F01"",""product_name"":""Jakobs...","DEFAULT:46,68",0.09
1312,173429537086439,16.76,,1752240410,626889,16.76,2.01,2024-12-15 21:43:00,adstrong CSS,https://preisvergleich.adstrong.de/,NaN,https://www.hagengrote.de/,https://www.google.com/,"DEFAULT:16,76",Germany,1: Hagen_Grote,"[{""product_id"":""027N18"",""product_name"":""Nachfü...","DEFAULT:16,76",0.03
1313,173429902461039,115.08,,1752301040,626889,115.08,13.81,2024-12-15 22:44:00,adstrong CSS,https://preisvergleich.adstrong.de/,NaN,https://www.hagengrote.de/,https://www.google.com/,"DEFAULT:115,08",Germany,1: Hagen_Grote,"[{""product_id"":""029Z03"",""product_name"":""Perfek...","DEFAULT:115,08",0.23


In [20]:

df_grouped['Fakturierte Nettoumsatz'] = df_grouped['Fakturierte Nettoumsatz'].fillna(0)

# 3. Create 'Difference' column
def calculate_difference(row):
    row['Gesamtumsatz'] = row['Gesamtumsatz'].astype(float)
    sale_betrag = row['Gesamtumsatz']
    fakturierte_nettoumsatz = row['Fakturierte Nettoumsatz']
    difference = sale_betrag - fakturierte_nettoumsatz
    return '' if round(difference, 0) == 0 else f"{round(difference, 2):.2f}"

# df_grouped['Difference'] = df_grouped.apply(calculate_difference, axis=1)
df_grouped.to_excel('morefire_Roxxad.xlsx', index=False)
